In [111]:
# data

refraction_NPK51 = {
    "R" : 1.5268,
    "G" : 1.5319,
    "B" : 1.5367,
    "Abbe" : 76.97
}
refraction_NKZFS4 = {
    "R" : 1.6098,
    "G" : 1.6202,
    "B" : 1.6300,
    "Abbe" : 44.49
}
refraction_SF15 = {
    "R" : 1.6932,
    "G" : 1.7105,
    "B" : 1.7280,
    "Abbe" : 30.07
}

foci = 200
n_air = 1.0003

In [115]:
import sympy as smp

f, f1, f2, f3, R1, R2, R3, V1, V2, V3, n1, n2, n3, nL, d1, d2, d3, na = smp.symbols(
    r'f, f_1, f_2, f_3, R_1, R_2, R_3, V_1, V_2, V_3, n_1, n_2, n_3, n_L, d_1, d_2, d_3, n_a')


balanceEq = smp.Eq(1/f, 1/f1 + 1/f2 + 1/f3)

lensmaker_eq_1 = smp.Eq(1 / f1, (n1 - 1) * (1 / R1 - 1 / R2) + ((n1 - 1)**2 * d1) / (n1 * R1 * R2))
lensmaker_eq_2 = smp.Eq(1 / f2, (n2 - 1) * (1 / R2 - 1 / R3) + ((n2 - 1)**2 * d2) / (n2 * R2 * R3))
lensmaker_eq_3 = smp.Eq(1 / f3, (n3 - 1) * (1 / R3))


balanceEq = balanceEq.subs({1/f1 : lensmaker_eq_1.rhs, 1/f2 : lensmaker_eq_2.rhs, 1/f3 : lensmaker_eq_3.rhs})
balanceEq

Eq(1/f_1, (-1/R_2 + 1/R_1)*(n_1 - 1) + d_1*(n_1 - 1)**2/(R_1*R_2*n_1))

Eq(1/f, (-1/R_2 + 1/R_1)*(n_1 - 1) + (-1/R_3 + 1/R_2)*(n_2 - 1) + (n_3 - 1)/R_3 + d_2*(n_2 - 1)**2/(R_2*R_3*n_2) + d_1*(n_1 - 1)**2/(R_1*R_2*n_1))

In [117]:
balanceEq_R = balanceEq.subs({f : foci, n1 : refraction_NPK51["R"], n2 : refraction_NKZFS4["R"], n3 : refraction_SF15["R"]})
balanceEq_G = balanceEq.subs({f : foci, n1 : refraction_NPK51["G"], n2 : refraction_NKZFS4["G"], n3 : refraction_SF15["G"]})
balanceEq_B = balanceEq.subs({f : foci, n1 : refraction_NPK51["B"], n2 : refraction_NKZFS4["B"], n3 : refraction_SF15["B"]})
balanceEq_B

Eq(1/200, 0.0980000000000001/R_3 + 0.0932999999999999/R_2 + 0.243496932515337*d_2/(R_2*R_3) + 0.5367/R_1 + 0.187445103143099*d_1/(R_1*R_2))

In [118]:
balanceEq_Abbe = smp.Eq(0, 1/(V1*f1) + 1/(V2*f2) + 1/(V3*f3))
balanceEq_Abbe = balanceEq_Abbe.subs({1/f1 : lensmaker_eq_1.rhs, 1/f2 : lensmaker_eq_2.rhs, 1/f3 : lensmaker_eq_3.rhs})

balanceEq_Abbe

Eq(0, ((-1/R_3 + 1/R_2)*(n_2 - 1) + d_2*(n_2 - 1)**2/(R_2*R_3*n_2))/V_2 + ((-1/R_2 + 1/R_1)*(n_1 - 1) + d_1*(n_1 - 1)**2/(R_1*R_2*n_1))/V_1 + (n_3 - 1)/(R_3*V_3))

In [119]:
balanceEq_Abbe_R = balanceEq_Abbe.subs({na : n_air, f : foci, n1 : refraction_NPK51["R"], n2 : refraction_NKZFS4["R"], n3 : refraction_SF15["R"], V1 : refraction_NPK51["Abbe"], V2 : refraction_NKZFS4["Abbe"], V3 : refraction_SF15["Abbe"]})
balanceEq_Abbe_G = balanceEq_Abbe.subs({na : n_air, f : foci, n1 : refraction_NPK51["G"], n2 : refraction_NKZFS4["G"], n3 : refraction_SF15["G"], V1 : refraction_NPK51["Abbe"], V2 : refraction_NKZFS4["Abbe"], V3 : refraction_SF15["Abbe"]})
balanceEq_Abbe_B = balanceEq_Abbe.subs({na : n_air, f : foci, n1 : refraction_NPK51["B"], n2 : refraction_NKZFS4["B"], n3 : refraction_SF15["B"], V1 : refraction_NPK51["Abbe"], V2 : refraction_NKZFS4["Abbe"], V3 : refraction_SF15["Abbe"]})
balanceEq_Abbe_B

Eq(0, 0.010049690753044/R_3 + 0.00718763893876387/R_2 + 0.00547307108373426*d_2/(R_2*R_3) + 0.00697284656359621/R_1 + 0.00243530080736779*d_1/(R_1*R_2))

In [145]:
from sympy.utilities.lambdify import lambdify

# Lambdify las ecuaciones
func_balanceEq_R = lambdify((R1, R2, R3, d1, d2), balanceEq_R.lhs - balanceEq_R.rhs, "numpy")
func_balanceEq_G = lambdify((R1, R2, R3, d1, d2), balanceEq_G.lhs - balanceEq_G.rhs, "numpy")
func_balanceEq_B = lambdify((R1, R2, R3, d1, d2), balanceEq_B.lhs - balanceEq_B.rhs, "numpy")
func_balanceEq_Abbe_R = lambdify((R1, R2, R3, d1, d2), balanceEq_Abbe_R.lhs - balanceEq_Abbe_R.rhs, "numpy")
func_balanceEq_Abbe_G = lambdify((R1, R2, R3, d1, d2), balanceEq_Abbe_G.lhs - balanceEq_Abbe_G.rhs, "numpy")

# Definir la función del sistema de ecuaciones para fsolve, con penalización por signos incorrectos
def equations(vars):
    R1, R2, R3, d1, d2 = vars

    # Penalización de signo incorrecto (valor grande si el signo no es el correcto)
    penalty = 1e6
    penalty_R1 = penalty if R1 <= 0 else 0
    penalty_R2 = penalty if R2 >= 0 else 0
    penalty_R3 = penalty if R3 <= 0 else 0
    penalty_d1 = penalty if d1 <= 0 else 0
    penalty_d2 = penalty if d2 <= 0 else 0

    return [
        func_balanceEq_R(R1, R2, R3, d1, d2) + penalty_R1 + penalty_R2 + penalty_R3 + penalty_d1 + penalty_d2,
        func_balanceEq_G(R1, R2, R3, d1, d2) + penalty_R1 + penalty_R2 + penalty_R3 + penalty_d1 + penalty_d2,
        func_balanceEq_B(R1, R2, R3, d1, d2) + penalty_R1 + penalty_R2 + penalty_R3 + penalty_d1 + penalty_d2,
        func_balanceEq_Abbe_R(R1, R2, R3, d1, d2) + penalty_R1 + penalty_R2 + penalty_R3 + penalty_d1 + penalty_d2,
        func_balanceEq_Abbe_G(R1, R2, R3, d1, d2) + penalty_R1 + penalty_R2 + penalty_R3 + penalty_d1 + penalty_d2
    ]

# Usar fsolve con valores iniciales
initial_guess = [200, -200, 300, 5, 2]  # Ajusta según expectativas
solution = fsolve(equations, initial_guess)

print("Numerical solution:", solution)

Numerical solution: [ 187.74016392 -221.53356031  303.86069443    4.07773007   51.70173698]


In [142]:
func_balanceEq_Abbe_R(187.74016392, -221.53356031,  303.86069443,    4.07773007,   51.70173698)

-3.2019429293078766e-05